# Handle Missing Values: Best Practices
A code block to deal with missings and inspect dataframes

**Helper functions**
1. df.info()
    * lists feature types and shape
2. list_missings()
    * lists share of missing entries per feature
3. handle_missings()
    * replaces np.nan for object-type and numeric features

In [2]:
# load data
import io #parse from url
import requests #load from url

# handle data
import pandas as pd
import numpy as np
import itertools # unordered pairwise combinations of list elements

# to display all the columns of the dataframe in the notebook
pd.pandas.set_option('display.max_columns', None)

## Load data
Either from URL or local directory

In [59]:
#load data from URL or revert to local copy
try:
    url = 'https://dataverse.harvard.edu/api/access/datafile/3352340?gbrecs=false'
    s = requests.get(url).content
    df_raw = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='\t')
    print("Loaded from URL")
except:
    path = 'C://Users//Philipp//GDrive//Projekter//Causal Forest Tutorial//data//grit//'
    df_raw = pd.read_csv(path+'Sample2_Data.tab', sep='\t')
    print("Loaded from local directory")
    
data = df_raw.copy()

Loaded from URL


In [118]:
#select features
X_features = ['grit', 'male', 'task_ability', 'raven', 'grit_survey1', 
 'belief_survey1', 'mathscore1', 'verbalscore1', 'risk', 'inconsistent']

X_raw = data[X_features]

## General info

In [12]:
print("General info about dataframe")
print("-"*40)
data.info(verbose=False)

General info about dataframe
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Columns: 52 entries, sample to raven
dtypes: float64(52)
memory usage: 609.1 KB


## List missings

In [119]:
def list_missings(data, N=20):
    """
    Lists N features with the highest share of missings
    INPUT: 
        data: dataframe
        N: number of features listed
    OUTPUT
        print list of N features with the highest missing share
    """
    # make a list of the variables that contain missing values
    vars_with_na = [var for var in data.columns if data[var].isnull().sum() > 0]
    missings = data[vars_with_na].isnull().mean().sort_values(ascending=False)#.iloc[:N]
    missings_avg = round(missings.mean(),4)*100
    
    # determine percentage of missing values
    print(f"Top {N} (of {len(data.columns)}) Variables containing missings")
    print(f"Average {missings_avg}% missings")
    print("-"*40)
    print(missings[:N])
    print()

list_missings(data, N=3)
list_missings(X_raw, N=5)

Top 3 (of 52) Variables containing missings
Average 12.72% missings
----------------------------------------
verbalscore3    0.480987
mathscore3      0.478986
age             0.247498
dtype: float64

Top 5 (of 10) Variables containing missings
Average 10.97% missings
----------------------------------------
risk              0.142762
belief_survey1    0.122748
task_ability      0.110073
raven             0.107405
verbalscore1      0.105404
dtype: float64



## Replace missings
### Categorical variables
* select columns of type == 'Object'
* print share of missing per variable 
* replace np.nan with 'Missing' (i.e. new category)

### Numeric variables
* replace np.nan with mode
* create dummy variable, which is 1 if np.nan present

In [122]:
def handle_missings(data, nan_string='Missing'):
    """
    Replace np.nan with either "Missing" for object type or mode for numeric type, including dummy indicator
    INPUT
        data: covariate dataframe (df)
        nan_string: string that replaces np.nan
    OUTPUT
        dataframe without missings
    """
    df = data.copy()
    
    ### HANDLE CATEGORICAL FEATURES
    # make a list of the categorical variables that contain missing values
    cat_with_na = [var for var in df.columns if df[var].isnull().sum() > 0 and df[var].dtypes == 'O']

    # print percentage of missing values per variable
    print("% of missing values per OBJECT variable")
    print("-"*40)
    print(df[cat_with_na].isnull().mean())
    print()

    # replace missing values with new label: "Missing"
    df[cat_with_na] = df[cat_with_na].fillna(nan_string)

    # check that we have no missing information in the engineered variables
    print("Sanity check:")
    check = len(df[cat_with_na].isnull().sum())
    if check > 0:
        print(f"There are stil {check} categorical variables with missings.\n")
    else: 
        print(f"Missings replaced with '{nan_string}' among categorical variables.\n")
        
    ### HANDLE NUMERIC FEATURES
    # make a list of the categorical variables that contain missing values
    num_with_na = [var for var in df.columns if df[var].isnull().sum() > 0 and df[var].dtypes != 'O']

    # print percentage of missing values per variable
    print("% of missing values per NUMERIC variable")
    print("-"*40)
    print(df.loc[:,num_with_na].isnull().mean())
    print()

    # replace engineer missing values as we described above
    for var in num_with_na:

        # calculate the mode using the train set
        mode_val = df[var].mode()[0]

        # add binary missing indicator (in train and test)
        df.loc[:,var+'_na'] = np.where(df[var].isnull(), 1, 0)

        # replace missing values by the mode
        df.loc[:,var] = df.loc[:,var].fillna(mode_val)

    # check that we have no missing information in the engineered variables
    print("Sanity check:")
    check = df[num_with_na].isnull().sum().max() 
    if check > 0:
        print(f"There are stil {check} categorical variables with missings.")
    else: 
        print(f"Missings replaced with mode among numeric variables.")
        
    return df

In [123]:
X = handle_missings(X_raw)

% of missing values per OBJECT variable
----------------------------------------
Series([], dtype: float64)

Sanity check:
Missings replaced with 'Missing' among categorical variables.

% of missing values per NUMERIC variable
----------------------------------------
task_ability      0.110073
raven             0.107405
grit_survey1      0.074049
belief_survey1    0.122748
mathscore1        0.105404
verbalscore1      0.105404
risk              0.142762
dtype: float64

Sanity check:
Missings replaced with mode among numeric variables.
